In [1]:
import pickle
# import omegaconf
import warnings
import pickle
import numpy as np
warnings.filterwarnings("ignore", category=DeprecationWarning)

from load_results import load_result_dataset
import pandas as pd
pn1 = 'full_fine_tuning_5epochs_edge_article1'
pn2 = 'full_fine_tuning_5epochs_article1'

final_data = load_result_dataset(pn1, pn2)

../results/full_fine_tuning_5epochs_article1/CLIP-convnext_base_w-laion_aesthetic-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_5epochs_article1/CLIP-convnext_base_w-laion2B-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_5epochs_article1/deit_small_patch16_224.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_5epochs_article1/robust_resnet50_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_5epochs_article1/vit_small_patch16_224.augreg_in21k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_5epochs_article1/convnext_base.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_5epochs_article1/resnet50.a1_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_5epochs_article1/robust_vit_base_patch16_224_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_5epochs_article1/vit_base_patch16_224.mae_uc-merc

In [2]:
df = pd.DataFrame(final_data)
for arch_size in ('small', 'medium', 'large'):
    df_curr = df[ df.model_size == arch_size ]
    nan_percentage = (df_curr.isna().sum().sum() / df_curr.size) * 100
    print("Percentage of NaN values, for {} architecture: {}".format(arch_size, nan_percentage) )
    

Percentage of NaN values, for small architecture: 0.4578754578754579
Percentage of NaN values, for medium architecture: 0.0
Percentage of NaN values, for large architecture: 0.7122507122507122


In [4]:
import pandas as pd
import yaml

df = pd.DataFrame(final_data)
# df = df[df["model_size"].isin([ 'medium', 'large' ])]
# df.to_csv("./{}.csv".format("result_with_missing_values_max"))


def get_restart_job(row):
    is_missing = lambda col: pd.isna(row[col])

    missing = {col: is_missing(col) for col in ['clean_acc', 'Linf_acc', 'L1_acc', 'L2_acc', 'common_acc']}

    if all(missing.values()):
        return "job1_hpo.sh"
    elif not missing['clean_acc'] and not missing['Linf_acc'] and missing['L1_acc']:
        return "job4_test_l1.sh"
    elif not missing['clean_acc'] and not missing['Linf_acc'] and not missing['L1_acc'] and missing['L2_acc']:
        return "job5_test_l2.sh"
    elif not missing['clean_acc'] and not missing['Linf_acc'] and not missing['L1_acc'] and not missing['L2_acc'] and missing['common_acc']:
        return "job6_test_common.sh"
    else:
        return None  # means no job needs to be restarted


df['restart_from'] = df.apply(get_restart_job, axis=1)

to_restart = df[df['restart_from'].notna()][['backbone', 'dataset', 'loss_function', 'restart_from', 'model_size']]
# to_restart.to_csv("./to_relaunch_max.csv")
to_restart

,backbone,dataset,loss_function,restart_from,model_size
108,convnext_base.fb_in22k_ft_in1k,caltech101,TRADES_v2,job4_test_l1.sh,large
111,coatnet_2_rw_224.sw_in12k_ft_in1k,caltech101,TRADES_v2,job4_test_l1.sh,large
112,coatnet_2_rw_224.sw_in12k,caltech101,TRADES_v2,job4_test_l1.sh,large
347,convnext_base.clip_laion2b_augreg_ft_in12k_in1k,caltech101,CLASSIC_AT,job4_test_l1.sh,large
359,coat_tiny.in1k,caltech101,CLASSIC_AT,job4_test_l1.sh,small


In [3]:
# Pivot the table to group by 'backbone' and 'loss_function' in rows and create a hierarchical structure with 'dataset' as column index

# df['total_acc'] = df['clean_acc'] + df['Linf_acc']  # Sum of scores per dataset

import pandas as pd

size = 2
df = pd.DataFrame(final_data)

nan_percentage = (df.isna().sum().sum() / df.size) * 100
print(f"Percentage of NaN values: {nan_percentage:.2f}%")

df = df[ df.model_size == size ]

grouped_df = df.pivot_table(
    index=['backbone', 'loss_function'],
    columns='dataset',
    # dropna=False
)

# Rename column levels
grouped_df.columns.set_names(["metric", "dataset"], inplace=True)

# Swap column levels → dataset becomes level 0, metric becomes level 1
grouped_df.columns = grouped_df.columns.swaplevel(0, 1)

# (Optional) Sort columns so that all metrics are grouped within each dataset
grouped_df = grouped_df.sort_index(axis=1, level=0)

# grouped_df.to_csv("./{}.csv".format(project_name) )

# --- existing shortcuts ----------------------------------------------------
sum_scores  = grouped_df.xs('sum',  level=1, axis=1)     # (rows × 6 datasets)
geom_scores = grouped_df.xs('geom', level=1, axis=1)     # (rows × 6 datasets)

#--- 1. add the TOTAL aggregates you already computed ----------------------
grouped_df[('TOTAL', 'score_sum')]  = sum_scores.sum(axis=1)
grouped_df[('TOTAL', 'score_geom')] = geom_scores.sum(axis=1)

# --- 2. count NaNs across datasets -----------------------------------------
grouped_df[('TOTAL', 'nan_sum_cnt')]  = sum_scores.isna().sum(axis=1)
grouped_df[('TOTAL', 'nan_geom_cnt')] = geom_scores.isna().sum(axis=1)

# --- 3. rank as before ------------------------------------------------------
df_sorted = grouped_df.sort_values(('TOTAL', 'score_sum'), ascending=False)

# Add ranks (1 = best/highest)
df_sorted[('TOTAL', 'rank_sum')]  = df_sorted[('TOTAL', 'score_sum')].rank(ascending=False, method='min')
df_sorted[('TOTAL', 'rank_geom')] = df_sorted[('TOTAL', 'score_geom')].rank(ascending=False, method='min')



df_sorted['TOTAL'].head(10)
# df_sorted.to_csv("./df_sorded_{}.csv".format(project_name) )


Percentage of NaN values: 10.83%


,metric,score_sum,score_geom,nan_sum_cnt,nan_geom_cnt,rank_sum,rank_geom
backbone,loss_function,,,,,,
swin_base_patch4_window7_224.ms_in22k_ft_in1k,TRADES_v2,7.269683,0.008315,0,0,1.0,4.0
convnext_base.fb_in1k,TRADES_v2,6.424079,0.000046,0,0,2.0,12.0
convnext_base.fb_in22k,TRADES_v2,5.775333,0.000586,0,0,3.0,9.0
eva02_base_patch14_224.mim_in22k,TRADES_v2,5.749286,0.000100,0,0,4.0,10.0
CLIP-convnext_base_w-laion_aesthetic-s13B-b82K,TRADES_v2,5.568825,0.019393,0,0,5.0,2.0
vit_base_patch16_clip_224.laion2b,TRADES_v2,4.898714,0.006836,0,0,6.0,6.0
CLIP-convnext_base_w-laion2B-s13B-b82K,TRADES_v2,4.634952,0.001137,0,0,7.0,8.0
vit_base_patch16_224.augreg_in1k,TRADES_v2,4.174905,0.000020,0,0,8.0,15.0
convnext_base.fb_in22k_ft_in1k,TRADES_v2,4.142571,0.000002,1,1,9.0,24.0


In [20]:
# Flatten the columns
df_sorted.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_sorted.columns]


df_reset = df_sorted.reset_index()
sub_df = df_reset[["backbone", "loss_function", "TOTAL_score_sum"]]

# Now access the columns

sub_df

,backbone,loss_function,TOTAL_score_sum
0,swin_base_patch4_window7_224.ms_in22k_ft_in1k,TRADES_v2,7.269683
1,convnext_base.fb_in1k,TRADES_v2,6.424079
2,convnext_base.fb_in22k,TRADES_v2,5.775333
3,eva02_base_patch14_224.mim_in22k,TRADES_v2,5.749286
4,vit_base_patch16_clip_224.laion2b,TRADES_v2,4.762714
5,CLIP-convnext_base_w-laion2B-s13B-b82K,TRADES_v2,4.634952
6,CLIP-convnext_base_w-laion_aesthetic-s13B-b82K,TRADES_v2,4.238667
7,vit_base_patch16_224.augreg_in1k,TRADES_v2,4.174905
8,convnext_base.fb_in22k_ft_in1k,TRADES_v2,4.142571
9,swin_base_patch4_window7_224.ms_in22k_ft_in1k,CLASSIC_AT,3.921143


In [21]:

from matplotlib import cm
from matplotlib.colors import to_hex
import re


import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px


import pandas as pd

df1 = grouped_df.copy()

metrics_to_sum = ["L1_acc", "L2_acc", "Linf_acc", "clean_acc", "common_acc"]

for metric in metrics_to_sum:
    metric_cols = [col for col in df1.columns if col[1] == metric]
    df1[f'sum_{metric}'] = df1[metric_cols].sum(axis=1)

df_per_row_sums = df1[[f'sum_{m}' for m in metrics_to_sum]]

df_per_row_sums.columns = df_per_row_sums.columns.get_level_values(0)

df_per_row_sums.columns.name = 'metric'

df_per_row_sums


# Your model type mapping
model_type_map = {
    'convnext_base': "fully convolutional",
    'convnext_tiny': "fully convolutional",
    'deit_small': "fully attention",
    'vit_base': "fully attention",
    'vit_small': "fully attention",
    'resnet50': "fully convolutional",
    'eva02_base': "fully attention",
    'eva02_tiny': "fully attention",
    'swin_base': "fully attention",
    'swin_tiny': "fully attention",
    'coatnet_0': "hybrid",
    'coatnet_2': "hybrid",
    'regnetx_004': "fully convolutional",
    'efficientnet-b0': "fully convolutional", 
    'deit_tiny': "fully attention",
    'mobilevit-small': "hybrid",
    'mobilenetv3': "fully convolutional",
    'edgenext_small': "fully convolutional",
    'coat_tiny': "hybrid",
}

# Colormap for each model type
type_to_cmap = {
    'fully convolutional': cm.Purples,     # deep violet → lavender
    'fully attention': cm.Oranges,         # strong orange → light peach
    'hybrid': cm.Greens                    # forest → mint
}

# Extract base backbone name from full backbone string
def extract_base_name(backbone_name):
    for base in model_type_map:
        if base in backbone_name:
            return base
    return 'unknown'

# Get all unique backbones
backbones = sorted(set(backbone for backbone, _ in df_per_row_sums.index))

# Map backbones to model type
backbone_model_type = {b: model_type_map.get(extract_base_name(b), 'unknown') for b in backbones}

# Assign gradient color per backbone based on model type
color_map = {}
for model_type in ['fully convolutional', 'fully attention', 'hybrid']:
    bks = [bk for bk in backbones if backbone_model_type[bk] == model_type]
    n = len(bks)
    for i, bk in enumerate(bks):
        start, end = 0.5, 0.9  # avoid very light or very dark edges
        position = start + (end - start) * (i / max(n - 1, 1))
        color = to_hex(type_to_cmap[model_type](position))
        color_map[bk] = color


# Define metrics
metrics = ['sum_L1_acc', 'sum_L2_acc', 'sum_Linf_acc', 'sum_clean_acc', 'sum_common_acc']

# Create a list of unique backbones (regardless of loss)
unique_backbones = sorted(set(backbone for backbone, _ in df_per_row_sums.index))

# # Assign a unique color to each backbone
# color_palette = px.colors.qualitative.Plotly
# color_map = dict(zip(unique_backbones, color_palette * (len(unique_backbones) // len(color_palette) + 1)))

# Define line styles per loss function
line_styles = {
    'CLASSIC_AT': 'solid',
    'TRADES_v2': 'dot'
}

# Create the radar plot

from plotly.colors import qualitative
color_palette = qualitative.Bold

# Get unique backbones
unique_backbones = sorted(set(backbone for backbone, _ in df_per_row_sums.index))

# Assign colors per backbone
# color_map = dict(zip(unique_backbones, color_palette * (len(unique_backbones) // len(color_palette) + 1)))

fig = go.Figure()

for (backbone, loss), row in df_per_row_sums.iterrows():
    fig.add_trace(go.Scatterpolar(
        r=row.tolist() + [row.tolist()[0]],  # Close the loop
        theta=metrics + [metrics[0]],
        mode='lines',
        name=f"{backbone} ({loss})",
        line=dict(
            width=2,
            color=color_map[backbone],
            dash=line_styles.get(loss, 'solid')
        )
    ))

fig.update_layout(
    width=500, 
    height=500,
    paper_bgcolor='white',
    plot_bgcolor='white',
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 6],
            showline=True,
            linewidth=1,
            linecolor="lightgrey",
            gridcolor="lightgrey",
            gridwidth=0.5,
        ),
        angularaxis=dict(
            direction="clockwise",
            showline=True,
            linewidth=1,
            linecolor="lightgrey",
            gridcolor="lightgrey",
            gridwidth=0.5,
        ),
        bgcolor='white'
    ),
    legend=dict(
        x=0,
        y=1.0,
        xanchor='left',
        yanchor='top',
        bgcolor='rgba(255,255,255,0.9)',
        bordercolor='lightgrey',
        borderwidth=1,
        font=dict(size=11),
    ),
    showlegend=False,
    font=dict(color='black'),
    margin=dict(l=25, r=25, t=25, b=25)  # <<< reduced margins
)



fig.show()

fig.write_image("./radar_plot_{}_{}.png".format(size, pn2), scale=3  )                # upscale for higher DPI (1 = default)

